# AI 생성 이미지 판별 - 탐색적 데이터 분석 (EDA)

이 노트북에서는 AI 생성 이미지와 실제 이미지 데이터셋을 탐색합니다.

## 목차
1. 라이브러리 임포트
2. 데이터 경로 설정 및 기본 정보
3. 클래스별 데이터 분포
4. 샘플 이미지 시각화
5. 이미지 크기 및 해상도 분석
6. 색상 분포 분석
7. 통계 요약

## 1. 라이브러리 임포트

In [ ]:
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# 랜덤 시드 고정
random.seed(42)
np.random.seed(42)

print("라이브러리 임포트 완료!")

## 2. 데이터 경로 설정 및 기본 정보

In [ ]:
# 프로젝트 루트 디렉토리
PROJECT_ROOT = Path('.').absolute().parent
DATA_DIR = PROJECT_ROOT / 'data' / 'raw'

FAKE_DIR = DATA_DIR / 'FAKE'
REAL_DIR = DATA_DIR / 'REAL'

print(f"프로젝트 루트: {PROJECT_ROOT}")
print(f"데이터 디렉토리: {DATA_DIR}")
print(f"\nFAKE 디렉토리 존재: {FAKE_DIR.exists()}")
print(f"REAL 디렉토리 존재: {REAL_DIR.exists()}")

In [ ]:
# 이미지 파일 목록 가져오기
fake_images = list(FAKE_DIR.glob('*.jpg')) + list(FAKE_DIR.glob('*.png'))
real_images = list(REAL_DIR.glob('*.jpg')) + list(REAL_DIR.glob('*.png'))

print(f"FAKE 이미지 개수: {len(fake_images):,}")
print(f"REAL 이미지 개수: {len(real_images):,}")
print(f"전체 이미지 개수: {len(fake_images) + len(real_images):,}")

## 3. 클래스별 데이터 분포

In [ ]:
# 데이터 분포 시각화
fig, ax = plt.subplots(figsize=(10, 6))

classes = ['FAKE', 'REAL']
counts = [len(fake_images), len(real_images)]
colors = ['#FF6B6B', '#4ECDC4']

bars = ax.bar(classes, counts, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# 막대 위에 값 표시
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height):,}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('이미지 개수', fontsize=12)
ax.set_title('클래스별 이미지 분포', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# 비율 출력
total = len(fake_images) + len(real_images)
print(f"\nFAKE 비율: {len(fake_images)/total*100:.2f}%")
print(f"REAL 비율: {len(real_images)/total*100:.2f}%")

## 4. 샘플 이미지 시각화

In [ ]:
# FAKE 이미지 샘플
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('FAKE (AI 생성) 이미지 샘플', fontsize=16, fontweight='bold', y=1.02)

sample_fake = random.sample(fake_images, 10)

for idx, (ax, img_path) in enumerate(zip(axes.flat, sample_fake)):
    img = Image.open(img_path)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"{img.size[0]}x{img.size[1]}", fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# REAL 이미지 샘플
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('REAL (실제 작품) 이미지 샘플', fontsize=16, fontweight='bold', y=1.02)

sample_real = random.sample(real_images, 10)

for idx, (ax, img_path) in enumerate(zip(axes.flat, sample_real)):
    img = Image.open(img_path)
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"{img.size[0]}x{img.size[1]}", fontsize=10)

plt.tight_layout()
plt.show()

## 5. 이미지 크기 및 해상도 분석

In [ ]:
# 이미지 크기 정보 수집 (샘플링)
print("이미지 크기 분석 중... (샘플: 각 클래스에서 1000개씩)")

def get_image_info(image_paths, sample_size=1000):
    """이미지 크기 정보 수집"""
    sampled_paths = random.sample(image_paths, min(sample_size, len(image_paths)))
    
    widths = []
    heights = []
    file_sizes = []
    
    for img_path in tqdm(sampled_paths, desc="이미지 분석"):
        try:
            img = Image.open(img_path)
            widths.append(img.size[0])
            heights.append(img.size[1])
            file_sizes.append(os.path.getsize(img_path) / 1024)  # KB
        except Exception as e:
            print(f"오류 발생: {img_path} - {e}")
            continue
    
    return widths, heights, file_sizes

# 각 클래스별 분석
fake_widths, fake_heights, fake_sizes = get_image_info(fake_images)
real_widths, real_heights, real_sizes = get_image_info(real_images)

print("\n분석 완료!")

In [ ]:
# 이미지 크기 분포 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('이미지 크기 및 파일 크기 분포', fontsize=16, fontweight='bold')

# FAKE - Width
axes[0, 0].hist(fake_widths, bins=50, color='#FF6B6B', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('FAKE - 이미지 너비 (Width)', fontweight='bold')
axes[0, 0].set_xlabel('Width (pixels)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(np.mean(fake_widths), color='red', linestyle='--', label=f'Mean: {np.mean(fake_widths):.0f}')
axes[0, 0].legend()

# FAKE - Height
axes[0, 1].hist(fake_heights, bins=50, color='#FF6B6B', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('FAKE - 이미지 높이 (Height)', fontweight='bold')
axes[0, 1].set_xlabel('Height (pixels)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(np.mean(fake_heights), color='red', linestyle='--', label=f'Mean: {np.mean(fake_heights):.0f}')
axes[0, 1].legend()

# FAKE - File Size
axes[0, 2].hist(fake_sizes, bins=50, color='#FF6B6B', alpha=0.7, edgecolor='black')
axes[0, 2].set_title('FAKE - 파일 크기', fontweight='bold')
axes[0, 2].set_xlabel('File Size (KB)')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].axvline(np.mean(fake_sizes), color='red', linestyle='--', label=f'Mean: {np.mean(fake_sizes):.0f} KB')
axes[0, 2].legend()

# REAL - Width
axes[1, 0].hist(real_widths, bins=50, color='#4ECDC4', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('REAL - 이미지 너비 (Width)', fontweight='bold')
axes[1, 0].set_xlabel('Width (pixels)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].axvline(np.mean(real_widths), color='blue', linestyle='--', label=f'Mean: {np.mean(real_widths):.0f}')
axes[1, 0].legend()

# REAL - Height
axes[1, 1].hist(real_heights, bins=50, color='#4ECDC4', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('REAL - 이미지 높이 (Height)', fontweight='bold')
axes[1, 1].set_xlabel('Height (pixels)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].axvline(np.mean(real_heights), color='blue', linestyle='--', label=f'Mean: {np.mean(real_heights):.0f}')
axes[1, 1].legend()

# REAL - File Size
axes[1, 2].hist(real_sizes, bins=50, color='#4ECDC4', alpha=0.7, edgecolor='black')
axes[1, 2].set_title('REAL - 파일 크기', fontweight='bold')
axes[1, 2].set_xlabel('File Size (KB)')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].axvline(np.mean(real_sizes), color='blue', linestyle='--', label=f'Mean: {np.mean(real_sizes):.0f} KB')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 6. 색상 분포 분석

In [ ]:
# 샘플 이미지의 색상 분포 분석
def analyze_color_distribution(image_paths, num_samples=100):
    """이미지의 RGB 색상 분포 분석"""
    sampled_paths = random.sample(image_paths, min(num_samples, len(image_paths)))
    
    r_values = []
    g_values = []
    b_values = []
    
    for img_path in tqdm(sampled_paths, desc="색상 분석"):
        try:
            img = Image.open(img_path).convert('RGB')
            img_array = np.array(img)
            
            # 이미지를 평탄화하여 색상 값 추출
            r_values.extend(img_array[:, :, 0].flatten())
            g_values.extend(img_array[:, :, 1].flatten())
            b_values.extend(img_array[:, :, 2].flatten())
        except Exception as e:
            continue
    
    return r_values, g_values, b_values

print("색상 분포 분석 중... (샘플: 각 100개씩)")
fake_r, fake_g, fake_b = analyze_color_distribution(fake_images, num_samples=100)
real_r, real_g, real_b = analyze_color_distribution(real_images, num_samples=100)
print("색상 분석 완료!")

In [ ]:
# 색상 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('RGB 색상 분포 비교', fontsize=16, fontweight='bold')

# FAKE 색상 분포
axes[0].hist(fake_r, bins=50, alpha=0.5, label='Red', color='red')
axes[0].hist(fake_g, bins=50, alpha=0.5, label='Green', color='green')
axes[0].hist(fake_b, bins=50, alpha=0.5, label='Blue', color='blue')
axes[0].set_title('FAKE - RGB 분포', fontweight='bold')
axes[0].set_xlabel('Pixel Value (0-255)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(alpha=0.3)

# REAL 색상 분포
axes[1].hist(real_r, bins=50, alpha=0.5, label='Red', color='red')
axes[1].hist(real_g, bins=50, alpha=0.5, label='Green', color='green')
axes[1].hist(real_b, bins=50, alpha=0.5, label='Blue', color='blue')
axes[1].set_title('REAL - RGB 분포', fontweight='bold')
axes[1].set_xlabel('Pixel Value (0-255)')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 통계 요약

In [ ]:
# 통계 요약 테이블
summary_data = {
    '클래스': ['FAKE', 'REAL'],
    '이미지 개수': [len(fake_images), len(real_images)],
    '평균 너비 (px)': [f"{np.mean(fake_widths):.0f}", f"{np.mean(real_widths):.0f}"],
    '평균 높이 (px)': [f"{np.mean(fake_heights):.0f}", f"{np.mean(real_heights):.0f}"],
    '평균 파일 크기 (KB)': [f"{np.mean(fake_sizes):.0f}", f"{np.mean(real_sizes):.0f}"],
    '최소 너비 (px)': [f"{np.min(fake_widths):.0f}", f"{np.min(real_widths):.0f}"],
    '최대 너비 (px)': [f"{np.max(fake_widths):.0f}", f"{np.max(real_widths):.0f}"],
    '최소 높이 (px)': [f"{np.min(fake_heights):.0f}", f"{np.min(real_heights):.0f}"],
    '최대 높이 (px)': [f"{np.max(fake_heights):.0f}", f"{np.max(real_heights):.0f}"]
}

summary_df = pd.DataFrame(summary_data)
print("\n=== 데이터셋 통계 요약 ===")
print(summary_df.to_string(index=False))

## 주요 발견 사항

### 데이터 분포
- FAKE와 REAL 클래스가 균형잡혀 있음 (각각 약 20,000개)
- 클래스 불균형 문제가 없어 학습에 유리

### 이미지 크기
- 이미지 크기가 다양함
- 모델 학습을 위해 224x224 또는 256x256으로 리사이징 필요

### 색상 분포
- FAKE와 REAL의 색상 분포에 차이가 있을 수 있음
- 이는 모델이 학습할 수 있는 중요한 특징

### 다음 단계
1. 데이터 전처리 (리사이징, 정규화)
2. 데이터 분할 (Train/Val/Test)
3. 데이터 증강 (Data Augmentation) 적용